In [1]:
import pandas as pd
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
data_train = pd.read_csv('https://code.s3.yandex.net/datasets/gold_recovery_train_new.csv', index_col='date',parse_dates=True)
data_test = pd.read_csv('https://code.s3.yandex.net/datasets/gold_recovery_test_new.csv', index_col='date',parse_dates=True)
data_full = pd.read_csv('https://code.s3.yandex.net/datasets/gold_recovery_full_new.csv', index_col='date',parse_dates=True)

In [3]:
missed_test_columns = set(list(data_train.columns.values))-set(list(data_test.columns.values))
missed_test_columns

{'final.output.concentrate_ag',
 'final.output.concentrate_au',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_au',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_au',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_sol',
 'rougher.calculation.au_pb_ratio',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_au',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_au',
 'rougher.output.ta

In [4]:
data_train = data_train.fillna(method='ffill')
data_test = data_test.fillna(method='ffill')
data_full = data_full.fillna(method='ffill')

In [5]:
data_full_merged = data_full[["rougher.output.recovery", "final.output.recovery", "rougher.output.concentrate_au", "rougher.output.concentrate_ag", "rougher.output.concentrate_pb", "rougher.output.concentrate_sol", "final.output.concentrate_au", "final.output.concentrate_ag", "final.output.concentrate_pb", "final.output.concentrate_sol"]]
data_test = data_test.merge(data_full_merged, on="date", how="left")

In [6]:
def calculate_smape(target_actual, target_predictions):
    numerator = np.abs(target_actual - target_predictions)
    denominator = (np.abs(target_actual) + np.abs(target_predictions)) / 2 
    return 1 / len(target_actual) * np.sum(numerator / denominator) * 100 
  
def calculate_final_smape(target_actual, target_predictions):
    target_actual_rougher = target_actual['rougher.output.recovery']
    target_actual_final = target_actual['final.output.recovery']
    target_predictions_rougher = target_predictions[:, 0]
    target_predictions_final = target_predictions[:, 1]
    return 0.25 * calculate_smape(target_actual_rougher, target_predictions_rougher) + 0.75 * calculate_smape(target_actual_rougher, target_predictions_rougher)

smape_scorer = make_scorer(calculate_final_smape, greater_is_better=False)


In [7]:
features_rough_concentrate = [
    'rougher.output.concentrate_au',
    'rougher.output.concentrate_ag',
    'rougher.output.concentrate_pb',
    'rougher.output.concentrate_sol',
]

features_input_concentrate = [
    'rougher.input.feed_au',
    'rougher.input.feed_ag',
    'rougher.input.feed_pb',
    'rougher.input.feed_sol',
]
features_final_concentrate = [
    'final.output.concentrate_au',
    'final.output.concentrate_ag',
    'final.output.concentrate_pb',
    'final.output.concentrate_sol',
]

display(data_test.shape)
display(data_train.shape)

data_train = data_train[data_train[features_input_concentrate].sum(1) > 0]
data_train = data_train[data_train[features_rough_concentrate].sum(1) > 0]
data_train = data_train[data_train[features_final_concentrate].sum(1) > 0]

data_full = data_full.loc[data_test.index]
data_test = data_test[data_full[features_input_concentrate].sum(1) > 0]
data_test = data_test[data_full[features_rough_concentrate].sum(1) > 0]
data_test = data_test[data_full[features_final_concentrate].sum(1) > 0]
display(data_test.shape)
display(data_train.shape)

(5290, 62)

(14149, 86)

/var/folders/nc/53ydtkz56jj83586jxv6mvlm0000gn/T/ipykernel_56354/250994860.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_test = data_test[data_full[features_final_concentrate].sum(1) > 0]


(5154, 62)

(13795, 86)

In [8]:
features_train = data_train.drop(columns=["rougher.output.recovery", "final.output.recovery"], axis=1)
target_train = data_train[["rougher.output.recovery", "final.output.recovery"]]

features_test = data_test.drop(columns=["rougher.output.recovery", "final.output.recovery"], axis=1)
target_test = data_test[["rougher.output.recovery", "final.output.recovery"]]

In [9]:
param_grid = {'max_depth': range(2, 3)}
model = DecisionTreeRegressor(random_state=12345)
search = GridSearchCV(model, param_grid, cv=5, scoring=smape_scorer)
search.fit(features_train, target_train)
print('Лучшее значение smape для дерева решений: {} при значениях гиперпараметров: {}'.format(-search.best_score_, search.best_params_))
best_dtr = -search.best_score_

Лучшее значение smape для дерева решений: 5.8711978698020015 при значениях гиперпараметров: {'max_depth': 2}
